In [ ]:
%run ../authentification.ipynb
import re

def xstr(s):
    if s is None:
        s="NoData"
        return s
    elif s=="":
        s="NoData"
        return s        
    else:
        s=str(s)
        s=re.sub('[^ a-zA-Z0-9]', '', s)
        #s=s.encode(encoding='UTF-8',errors='strict')
    return str(s).replace('"', '')

In [ ]:
# Récupération des followers de trump, code validé sans doublons

followers = tweepy.Cursor(api.followers, id=user.id).items(4000)

while True:
    try:
        follower = next(followers)
        sgraph.run('MERGE (trump:User {id:"' + xstr(user.id) + '"'
                               ',screen_name:"'+xstr(user.screen_name)+'"'
                               ',url:"'+xstr(user.url)+'"'
                               ',description:"'+xstr(user.description)+'"'
                               ',created_at:"'+xstr(user.created_at)+'"'
                               ',geo_enabled: "'+xstr(user.geo_enabled)+'"'
                                ', name:"'+xstr(user.name)+'"})'
                           'MERGE (follower:User {id: "'+xstr(follower.id)+'"'
                               ', screen_name: "'+xstr(follower.screen_name)+'"'
                               ', url: "'+xstr(follower.url)+'"'
                               ', description: "'+xstr(follower.description)+'"'
                               ', created_at: "'+xstr(follower.created_at) +'"'
                               ', geo_enabled: "'+xstr(follower.geo_enabled)+'"'
                                ', name: "'+xstr(follower.name)+'"})'
                               ' CREATE UNIQUE (trump)-[:ISFOLLOWEDBY]->(follower)')
                               
        sgraph.run('MATCH (user:User) WHERE user.id="'+xstr(user.id)+'" SET user.followers_count = "'+xstr(user.followers_count)+'", user.friends_count="'+xstr(user.friends_count)+'", user.favourites_count="'+xstr(user.favourites_count)+'", user.statuses_count="'+xstr(user.statuses_count)+'", user.listed_count="'+xstr(user.listed_count)+'"')
        sgraph.run('MATCH (follower:User) WHERE follower.id="'+xstr(follower.id)+'" SET follower.followers_count = "'+str(follower.followers_count)+'", follower.friends_count="'+xstr(follower.friends_count)+'", follower.favourites_count="'+xstr(follower.favourites_count)+'", follower.statuses_count="'+xstr(follower.statuses_count)+'", follower.listed_count="'+xstr(follower.listed_count)+'"')   
    
        sgraph.run('MERGE (follower:User {id: "'+xstr(follower.id)+'"})' 
                   'MERGE (location:Location {location:"'+ xstr(follower.location) +'"})'
                   'CREATE UNIQUE (follower)-[:COMESFROM]->(location)')
    
    except tweepy.TweepError:
        print(str(tweepy.TweepError) + "Laissez moi me reposer pendant 16 minutes")
        time.sleep(60 * 16)
        print("on continue!")
        continue
    except StopIteration:
        break
                

In [ ]:
# Récupération des amis de Trump(personnes qu'il follow), Code Validé

friends = tweepy.Cursor(api.friends, id=user.id).items()
#
while True:
    try:
        friend = next(friends)
        sgraph.run('MERGE (trump:User {id:"' + xstr(user.id) + '"'
                               ', screen_name:"'+xstr(user.screen_name)+'"'
                               ', url:"'+xstr(user.url)+'"'
                               ', description:"'+xstr(user.description)+'"'
                               ', created_at:"'+xstr(user.created_at)+'"'
                               ', geo_enabled: "'+xstr(user.geo_enabled)+'"'
                                ', name:"'+xstr(user.name)+'"})'
                           'MERGE (friend:User {id: "'+xstr(friend.id)+'"'
                               ', screen_name: "'+xstr(friend.screen_name)+'"'
                               ', url: "'+xstr(friend.url)+'"'
                               ', description: "'+xstr(friend.description)+'"'
                               ', created_at: "'+xstr(friend.created_at) +'"'
                               ', geo_enabled: "'+xstr(friend.geo_enabled)+'"'
                                ', name: "'+xstr(friend.name)+'"})'
                               ' CREATE UNIQUE (trump)-[:ISFRIENDWITH]->(friend)')
        
        sgraph.run('MATCH (user:User) WHERE user.id="'+xstr(user.id)+'" SET user.followers_count = "'+xstr(user.followers_count)+'", user.friends_count="'+xstr(user.friends_count)+'", user.favourites_count="'+xstr(user.favourites_count)+'", user.statuses_count="'+xstr(user.statuses_count)+'", user.listed_count="'+xstr(user.listed_count)+'"')
        sgraph.run('MATCH (friend:User) WHERE friend.id="'+xstr(friend.id)+'" SET friend.followers_count = "'+xstr(friend.followers_count)+'", friend.friends_count="'+xstr(friend.friends_count)+'", friend.favourites_count="'+xstr(friend.favourites_count)+'", friend.statuses_count="'+xstr(friend.statuses_count)+'", friend.listed_count="'+xstr(friend.listed_count)+'"')   

        sgraph.run('MERGE (friend:User {id: "'+xstr(friend.id)+'"})' 
                   'MERGE (location:Location {location:"'+ xstr(friend.location) +'"})'
                   'CREATE UNIQUE (friend)-[:COMESFROM]->(location)')
        
        
    except tweepy.TweepError:
        print(str(tweepy.TweepError) + "Laissez moi me reposer pendant 16 minutes")
        time.sleep(60 * 16)
        print("on continue!")
        continue
    except StopIteration:
        break


In [ ]:
#Récupération des tweets de trump

tweets=tweepy.Cursor(api.user_timeline, screen_name=user.screen_name,tweet_mode='extended').items(5000)

while True:
    try:
        tweet=next(tweets)
        hashtags=tweet.entities.get("hashtags")
        user_mentions=tweet.entities.get("user_mentions")
        
                           
        if hasattr(tweet, 'coordinates'):
            coordinatestweet=tweet.coordinates
        else:
            coordinatestweet="NoData"
            
                 
        if tweet.place:
            placetweet=tweet.place.name
        else:
            placetweet="NoData"



        sgraph.run('match (trump:User {id: "'+xstr(user.id)+'"})'
                   'MERGE (tweet:Tweet {tid:"' + xstr(tweet.id) + '"'
                   ',source:"'+xstr(tweet.source)+'"'
                   ',coordinates:"'+xstr(coordinatestweet)+'"'
                   ',place:"'+xstr(placetweet)+'"'
                   ',date:"'+ str(tweet.created_at)+'"'
                   ',text: "'+xstr(tweet.full_text)+'"})'
                   'Create unique (trump)-[:POSTED]->(tweet)')
        
        sgraph.run('MATCH (t:Tweet) WHERE t.tid="'+xstr(tweet.id)+'" SET t.retweet_count = "'+xstr(tweet.retweet_count)+'", t.favorite_count="'+xstr(tweet.favorite_count)+'"')
        #sgraph.run('MATCH (u:User) WHERE u.id="'+xstr(status.user.id)+'" SET u.followers_count = "'+str(status.user.followers_count)+'", u.friends_count="'+str(status.user.friends_count)+'", u.statuses_count="'+str(status.user.statuses_count)+'"')
        
        for h in hashtags:
            
            sgraph.run(' Match (tweet:Tweet{tid: "'+xstr(tweet.id)+'"})'
                       ' MERGE (h:Hashtag{tag: "'+xstr(h['text'])+'",tid: "'+xstr(tweet.id)+'"})'
                       'CREATE unique (tweet)-[:HAS]->(h)')
          
        for m in user_mentions:
            
            sgraph.run(' Match (tweet:Tweet{tid: "'+xstr(tweet.id)+'"})'                      
                       ' MERGE (m:User{id: "'+xstr(m['id'])+'",name: "'+xstr(m['name'])+'",screen_name: "'+xstr(m['screen_name'])+'"})'
                       'CREATE unique (tweet)-[:MENTIONS]->(m)')
            
            
            
    except tweepy.TweepError:
        print(str(tweepy.TweepError) + "Laissez moi me reposer pendant 16 minutes")
        time.sleep(60 * 16)
        print("on continue!")
        continue
    except StopIteration:
        break

In [ ]:
# récupération des commentaires 

replies=tweepy.Cursor(api.search,q='to:realDonaldTrump',tweet_mode='extended').items(8000)  


while True:
    try:
        reply=next(replies)        
        
        if reply.in_reply_to_status_id is not None:   
            if reply.lang =="en":
                                           
                if hasattr(reply, 'coordinates'):
                    coordinatestweet=reply.coordinates
                else:
                    coordinatestweet="NoData"

                if reply.place:
                    placetweet=reply.place.name
                else:
                    placetweet="NoData"
                   
                sgraph.run('match (tweet:Tweet {tid:"'+ xstr(reply.in_reply_to_status_id) + '"})'
                       'MERGE (r:Tweet {tid:"' + xstr(reply.id) + '"'
                       ',date:"'+str(reply.created_at)+'"'
                       ',text: "'+xstr(reply.full_text)+'"'
                        ',source:"'+xstr(reply.source)+'"'
                       ',coordinates:"'+xstr(coordinatestweet)+'"'
                       ',place:"'+xstr(placetweet)+'"})'           
                       'Create UNIQUE (r)-[:REPLYTO]->(tweet)')
            
                sgraph.run('MATCH (r:Tweet) WHERE r.tid="'+xstr(reply.id)+'" SET r.retweet_count = "'+xstr(reply.retweet_count)+'", r.favorite_count="'+xstr(reply.favorite_count)+'"')     
                
                sgraph.run('MATCH (r:Tweet) WHERE r.tid="'+xstr(reply.id)+'"'
                           'MERGE (replyer:User {id: "'+xstr(reply.user.id)+'"'
                                           ', screen_name: "'+xstr(reply.user.screen_name)+'"'
                                           ', url: "'+xstr(reply.user.url)+'"'
                                           ', description: "'+xstr(reply.user.description)+'"'
                                           ', created_at: "'+xstr(reply.user.created_at) +'"'
                                           ', geo_enabled: "'+xstr(reply.user.geo_enabled)+'"'
                                            ', name: "'+xstr(reply.user.name)+'"})'
                                           ' CREATE UNIQUE (replyer)-[:POSTED]->(r)')
                
                sgraph.run('MERGE (replyer:User {id: "'+xstr(reply.user.id)+'"})' 
                   'MERGE (location:Location {location:"'+ xstr(reply.user.location) +'"})'
                   'CREATE UNIQUE (replyer)-[:COMESFROM]->(location)')
                    
                '''
                for h in hashtags:
            
                    sgraph.run(' Match (reply:Tweet{tid: "'+xstr(reply.id)+'"})'
                       ' MERGE (h:Hashtag{tag: "'+h['text']+'",tid: "'+xstr(reply.id)+'"})'
                       ' CREATE unique (reply)-[:HAS]->(h)')
          
                for m in user_mentions:
                    sgraph.run(' Match (reply:Tweet{tid: "'+xstr(reply.id)+'"})'                      
                       ' MERGE (m:User{id: "'+xstr(m['id'])+'",name: "'+xstr(m['name'])+'",screen_name: "'+xstr(m['screen_name'])+'"})'
                       ' CREATE unique (reply)-[:MENTIONS]->(m)')
                 '''
    except tweepy.TweepError:
        print(str(tweepy.TweepError) + "Laissez moi me reposer pendant 16 minutes")
        time.sleep(60 * 16)
        print("on continue!")
        continue
    except StopIteration:
        break